::## ANI Model Training for Metal



### Step 1: Installing TorchANI

First, we need to install TorchANI. Since the different cell size data are generated with the PBC conditions. During the data loading, this should appropriately be handled.

The original TorchANI code does not provide a function that can handle different cell sizes during the training.

We need to download a customized version for this tutorial.



In [1]:
!pip install git+https://github.com/gsjung0419/torchani_gs.git@min-cell

  Cloning https://github.com/gsjung0419/torchani_gs.git (to revision min-cell) to /tmp/pip-req-build-rpiflsdw
  Running command git clone --filter=blob:none --quiet https://github.com/gsjung0419/torchani_gs.git /tmp/pip-req-build-rpiflsdw
  Running command git checkout -b min-cell --track origin/min-cell
  Switched to a new branch 'min-cell'
  Branch 'min-cell' set up to track remote branch 'min-cell' from 'origin'.
  Resolved https://github.com/gsjung0419/torchani_gs.git to commit a941e6b79fc39e5ff305fcdbfd01c52f4cbbb6cf
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done


### Step 2: Importing Required Libraries

In this step, we import the necessary libraries for our task.

Also, let's utilize GPU.

In the Runtime menu, click 'change runtime type', you may need to restart the session and download the torchani again.

In [4]:
import torch
import torchani
import os
import math
import torch.utils.tensorboard
import tqdm

# helper function to convert energy unit from Hartree to kcal/mol
from torchani.units import hartree2kcalmol
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Check the device is cuda, otherwier, it take time", device)

Check the device is cuda, otherwier, it take time cuda


### Step 3: Loading the Dataset

I recommand to use your own google drive to setup your local drive to download files and save the log file for running.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Please decide where to run. You should provide the path of google drive where you are running this tutorial.

In [6]:
%cd /content/drive/MyDrive/ANI_Tutorials/Metal

/content/drive/MyDrive/ANI_Tutorials/Metal


In [7]:
try:
    path = os.path.dirname(os.path.realpath(__file__))
except NameError:
    path = os.getcwd()

#This file defines the energy shift for each atom species
sae_file = os.path.join(path, 'sae_linfit_dftb.data')

#This file defines the BP symmetry functions
const_file = os.path.join(path, 'rC.params')

consts = torchani.neurochem.Constants(const_file)
aev_computer = torchani.AEVComputer(**consts)

#Set up the minimum size of cell in your training data.
#This is based on the customized version. A better version for handling virial stress terms will be opened soon.
min_cell=torch.tensor([[10.0,0,0],[0,10.0,0],[0,0,10.0]],requires_grad=True,dtype=torch.float64,device=device)
aev_computer.setMinCell(min_cell)

#Note that the atom is set to 'C' for the simplicity but you can change it to any atom name.
#This is a just name, does not affect your trainning.
#But once you change it, following code should be consistent.
energy_shifter = torchani.neurochem.load_sae(sae_file)
species_order = ['C']

### Step 4: Data Preprocessing

In this step, we define the batchsize.

You can change and test.  

Training/Validation sets are already given

In [8]:
trpath = os.path.join(path, 'train.h5')
valpath = os.path.join(path, 'validation.h5')
batch_size = 64 #second epoch

Please install hdf5-tools to check the data "*.h5"

In [9]:
!apt-get install -y hdf5-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
hdf5-tools is already the newest version (1.10.7+repack-4ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [10]:
!h5ls -rf train.h5

/                        Group
/mols                    Group
/mols/AC0                Group
/mols/AC0/cell           Dataset {68, 3, 3}
/mols/AC0/coordinates    Dataset {68, 32, 3}
/mols/AC0/energies       Dataset {68}
/mols/AC0/forces         Dataset {68, 32, 3}
/mols/AC0/species        Dataset {32}
/mols/AC0/virial         Dataset {68, 6}
/mols/AC1                Group
/mols/AC1/cell           Dataset {136, 3, 3}
/mols/AC1/coordinates    Dataset {136, 32, 3}
/mols/AC1/energies       Dataset {136}
/mols/AC1/forces         Dataset {136, 32, 3}
/mols/AC1/species        Dataset {32}
/mols/AC1/virial         Dataset {136, 6}
/mols/AC2                Group
/mols/AC2/cell           Dataset {59, 3, 3}
/mols/AC2/coordinates    Dataset {59, 32, 3}
/mols/AC2/energies       Dataset {59}
/mols/AC2/forces         Dataset {59, 32, 3}
/mols/AC2/species        Dataset {32}
/mols/AC2/virial         Dataset {59, 6}
/mols/AC3                Group
/mols/AC3/cell           Dataset {116, 3, 3}
/mols/AC3/c

Data loading for batch trainning.

Note that additional_properties you can define to load it in addition to 'energy,'  and 'coordinates'.

In [11]:
training = torchani.data.load(
    trpath,
    additional_properties=('forces','cell')
).subtract_self_energies(energy_shifter, species_order).species_to_indices(species_order).shuffle()

validation = torchani.data.load(
    valpath,
    additional_properties=('forces','cell')
).subtract_self_energies(energy_shifter, species_order).species_to_indices(species_order).shuffle()

training = training.collate(batch_size).cache()
validation = validation.collate(batch_size).cache()

Here, we define PBC and AEV dimensions and network

In [12]:
pbc = torch.tensor([1,1,1],dtype=torch.bool,device=device)
aev_dim = aev_computer.aev_length

#Note that the original ANI potential utilizes CELU for the activation function.
#From the tests, we found that GELU works better for force and stress.
C_network = torch.nn.Sequential(
    torch.nn.Linear(aev_dim, 224),
    torch.nn.GELU(),
    torch.nn.Linear(224, 192),
    torch.nn.GELU(),
    torch.nn.Linear(192, 160),
    torch.nn.GELU(),
    torch.nn.Linear(160, 1)
)

#Define Neural Network
nn = torchani.ANIModel([C_network])


Here, we initialize all the parameters.

In [13]:
def init_params(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight, a=1.0)
        torch.nn.init.zeros_(m.bias)

nn.apply(init_params)

ANIModel(
  (0): Sequential(
    (0): Linear(in_features=54, out_features=224, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=224, out_features=192, bias=True)
    (3): GELU(approximate='none')
    (4): Linear(in_features=192, out_features=160, bias=True)
    (5): GELU(approximate='none')
    (6): Linear(in_features=160, out_features=1, bias=True)
  )
)

In [14]:
model = torchani.nn.Sequential(aev_computer, nn).to(device)

### Step 7: Training the Model

Here we define scheduler.

For the weights, we utilize Adam Optimizer.
For the bias, Stochastic Gradient Descendent (SGD) Algorithm is utilized.

In [15]:
AdamW = torch.optim.AdamW([
    # C networks
    {'params': [C_network[0].weight]},
    {'params': [C_network[2].weight], 'weight_decay': 0.00001},
    {'params': [C_network[4].weight], 'weight_decay': 0.000001},
    {'params': [C_network[6].weight]},
])

SGD = torch.optim.SGD([
    # C networks
    {'params': [C_network[0].bias]},
    {'params': [C_network[2].bias]},
    {'params': [C_network[4].bias]},
    {'params': [C_network[6].bias]},
], lr=1e-4)

#This schedule include the learning rate change.
#In this example, you may not observe learning-rate chages.
AdamW_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(AdamW, factor=0.5, patience=100, threshold=0)
SGD_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(SGD, factor=0.5, patience=100, threshold=0)

Define check points and the file name for the parameters.

In [16]:
latest_checkpoint = 'force-training-latest.pt'
if os.path.isfile(latest_checkpoint):
    checkpoint = torch.load(latest_checkpoint)
    nn.load_state_dict(checkpoint['nn'])
    AdamW.load_state_dict(checkpoint['AdamW'])
    SGD.load_state_dict(checkpoint['SGD'])
    AdamW_scheduler.load_state_dict(checkpoint['AdamW_scheduler'])
    SGD_scheduler.load_state_dict(checkpoint['SGD_scheduler'])

Define the validate() function to measure the performance of the trained model for the validation sets.

In [41]:
def validate():
    # run validation
    mse_sum = torch.nn.MSELoss(reduction='sum')
    total_mse = 0.0
    count = 0
    total_msef =0.0

    model.train(False)
    for properties in validation:
      species = properties['species'].to(device)
      #coordinates = properties['coordinates'].to(device).float()
      coordinates = properties['coordinates'].to(device).float().requires_grad_(True)
      true_energies = properties['energies'].to(device).float()
      true_forces = properties['forces'].to(device).float()
      cell = properties['cell'].to(device).float()
      _, predicted_energies = model((species, coordinates),cell,pbc)
      forces = -torch.autograd.grad(predicted_energies.sum(), coordinates)[0]
      total_mse += mse_sum(predicted_energies, true_energies).item()
      total_msef += mse_sum(true_forces, forces).item()
      count += predicted_energies.shape[0]


    model.train(True)
    return hartree2kcalmol(math.sqrt(total_mse / count)),hartree2kcalmol(math.sqrt(total_msef/count))

Define the tensorboard to write the log file to check the trainning.

In [18]:
tensorboard = torch.utils.tensorboard.SummaryWriter()

Here, we deifne the loss function.

The force components for the training is set 0.1.

Only RMSE of energy was check for the best model.

There can be different setting you may try to find.

In [ ]:
!rm *.pt
!ls

rm: cannot remove '*.pt': No such file or directory
ANI_Training.ipynb     rC.params  sae_linfit_dftb.data	validation.h5
nnp_training_force.py  runs	  train.h5


In [42]:
mse = torch.nn.MSELoss(reduction='none')

print("training starting from epoch", AdamW_scheduler.last_epoch + 1)

#Define the maximum number of epoch
max_epochs = 303

#Early_stopping but usually not activated.
early_stopping_learning_rate = 1.0E-5

force_coefficient = 0.2  # controls the importance of energy loss vs force loss

#The best model check point
best_model_checkpoint = 'force-training-best.pt'

for _ in range(AdamW_scheduler.last_epoch + 1, max_epochs):
    rmse,rmse_f = validate()

    print('RMSE of Energy and Forces ', rmse, rmse_f, 'at epoch', AdamW_scheduler.last_epoch + 1)

    learning_rate = AdamW.param_groups[0]['lr']

    if learning_rate < early_stopping_learning_rate:
        break

    # checkpoint
    if AdamW_scheduler.is_better(rmse, AdamW_scheduler.best):
        torch.save(nn.state_dict(), best_model_checkpoint)

    AdamW_scheduler.step(rmse)
    SGD_scheduler.step(rmse)

    # This allows us to check the history of training by tensorboad
    tensorboard.add_scalar('validation_rmse', rmse, AdamW_scheduler.last_epoch)
    tensorboard.add_scalar('best_validation_rmse', AdamW_scheduler.best, AdamW_scheduler.last_epoch)
    tensorboard.add_scalar('learning_rate', learning_rate, AdamW_scheduler.last_epoch)


    # Besides being stored in x, species and coordinates are also stored in y.
    # So here, for simplicity, we just ignore the x and use y for everything.
    for i, properties in tqdm.tqdm(
        enumerate(training),
        total=len(training),
        desc="epoch {}".format(AdamW_scheduler.last_epoch)
    ):
        species = properties['species'].to(device)
        coordinates = properties['coordinates'].to(device).float().requires_grad_(True)
        true_energies = properties['energies'].to(device).float()
        true_forces = properties['forces'].to(device).float()
        cell = properties['cell'].to(device).float()
        num_atoms = (species >= 0).sum(dim=1, dtype=true_energies.dtype)
        _, predicted_energies = model((species, coordinates),cell,pbc)

        # We can use torch.autograd.grad to compute force. Remember to
        # create graph so that the loss of the force can contribute to
        # the gradient of parameters, and also to retain graph so that
        # we can backward through it a second time when computing gradient
        # w.r.t. parameters.
        forces = -torch.autograd.grad(predicted_energies.sum(), coordinates, create_graph=True, retain_graph=True)[0]

        # Now the total loss has two parts, energy loss and force loss
        energy_loss = (mse(predicted_energies, true_energies) / num_atoms.sqrt()).mean()
        force_loss = (mse(true_forces, forces).sum(dim=(1, 2)) / num_atoms).mean()
        loss = energy_loss + force_coefficient * force_loss

        AdamW.zero_grad()
        SGD.zero_grad()
        loss.backward()
        AdamW.step()
        SGD.step()

        # write current batch loss to TensorBoard
        tensorboard.add_scalar('Energy_loss',energy_loss,AdamW_scheduler.last_epoch)
        tensorboard.add_scalar('Force_loss',force_loss,AdamW_scheduler.last_epoch)
        tensorboard.add_scalar('batch_loss',loss, AdamW_scheduler.last_epoch * len(training) + i)

    torch.save({
        'nn': nn.state_dict(),
        'AdamW': AdamW.state_dict(),
        'SGD': SGD.state_dict(),
        'AdamW_scheduler': AdamW_scheduler.state_dict(),
        'SGD_scheduler': SGD_scheduler.state_dict(),
    }, latest_checkpoint)

training starting from epoch 303


### Step 8: Evaluating the Model

After training, we evaluate the model's performance on the validation set.

In [43]:
def LoadANI(path,mname):
    sae_file = os.path.join(path, 'sae_linfit_dftb.data')  # noqa: E501
    const_file = os.path.join(path, 'rC.params')

    consts = torchani.neurochem.Constants(const_file)
    aev_computer = torchani.AEVComputer(**consts)

    #min_cell=torch.tensor([[10.0,0,0],[0,10.0,0],[0,0,10.0]],requires_grad=True,dtype=torch.float64,device=device)
    min_cell=torch.tensor([[10.0,0,0],[0,10.0,0],[0,0,10.0]],dtype=torch.float64,device=device)
    pbc = torch.tensor([1,1,1],dtype=torch.bool,device=device)
    aev_computer.setMinCell(min_cell)

    energy_shifter = torchani.neurochem.load_sae(sae_file)
    species_order = ['C']
    aev_dim = aev_computer.aev_length
    C_network = torch.nn.Sequential(
        torch.nn.Linear(aev_dim, 224),
        torch.nn.GELU(),
        torch.nn.Linear(224, 192),
        torch.nn.GELU(),
        torch.nn.Linear(192, 160),
        torch.nn.GELU(),
        torch.nn.Linear(160, 1)
    )

    nn = torchani.ANIModel([C_network])
    ptname = mname+'.pt'
    #nn.load_state_dict(torch.load('force-training-best.pt',map_location='cpu'))
    nn.load_state_dict(torch.load(ptname,map_location='cpu'))
    model = torchani.nn.Sequential(aev_computer, nn, energy_shifter).to(device)

    return model

In [44]:
bestmodel = LoadANI('./','force-training-best')

In [45]:
def evaluate(dataset):
    # run validation
    mse_sum = torch.nn.L1Loss(reduction='sum')
    total_mse = 0.0
    count = 0

    model.train(False)
    with torch.no_grad():
        for properties in dataset:
            species = properties['species'].to(device)
            coordinates = properties['coordinates'].to(device).float()
            true_energies = properties['energies'].to(device).float()
            true_forces = properties['forces'].to(device).float()
            cell = properties['cell'].to(device).float()
            _, predicted_energies = bestmodel((species, coordinates),cell,pbc)

            total_mse += mse_sum(predicted_energies, true_energies).item()
            count += predicted_energies.shape[0]

    model.train(True)
    return hartree2kcalmol(total_mse / count)

In [48]:
MAE=evaluate(training)*0.043/32*1000 # divided by the number of atoms. the unit to meV/atom
print("The error of training set (meV/atom)", MAE)
MAE=evaluate(validation)*0.043/32*1000 # divided by the number of atoms. the unit to meV/atom
print("The error of validation set (meV/atom)", MAE)

The error of training set (meV/atom) 3.2101794949590556
The error of validation set (meV/atom) 1.6875436915885036



If you think this tutorial is useful, consider cite the following references:

References
1. The function for different cell sizes was developed in this study:

  Jung, G. S., Myung, H., & Irle, S. (2023). Artificial neural network potentials for mechanics and fracture dynamics of two-dimensional crystals. Machine Learning: Science and Technology, 4(3), 035001.

2. The data set were prepared by the study:

 Jung, G. S., Lee, S., & Choi, J. Y. (2023). Data Distillation for Neural Network Potentials toward Foundational Dataset. arXiv preprint arXiv:2311.05407.

 using active learning tool developed by the study:
 Jung, G. S., Choi, J. Y., & Lee, S. M. (2024). Active learning of neural network potentials for rare events. Digital Discovery.

3. The original code of TorchANI developed by the study:

  Gao, X., Ramezanghorbani, F., Isayev, O., Smith, J. S., & Roitberg, A. E. (2020). TorchANI: A free and open source PyTorch-based deep learning implementation of the ANI neural network potentials. Journal of chemical information and modeling, 60(7), 3408-3415.



Load best model for validation set.